In [1]:
import sys
import logging
from optparse import OptionParser
from time import time
from datetime import datetime
from tqdm import tqdm
import json

import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.ensemble import GradientBoostingRegressor
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import psycopg2

/home/mc/anaconda3/envs/datascience/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
final_etsy_lda_output          = joblib.load('final_etsy_lda_output.pkl')
final_etsy_lda_model           = joblib.load('final_etsy_lda_model.pkl')
final_etsy_vectorizer          = joblib.load('final_etsy_vectorizer.pkl')
final_etsy_vectorized_features = joblib.load('final_etsy_vectorized_features.pkl')
final_etsy_data                = joblib.load('final_etsy_data.pkl')

In [6]:
pyLDAvis.sklearn.prepare(final_etsy_lda_model, final_etsy_vectorized_features, final_etsy_vectorizer, mds='mmds')

/home/mc/anaconda3/envs/datascience/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
5      0.421438  0.317928       1        1  4.823539
35    -0.083840 -0.476216       2        1  3.839419
34    -0.050760 -0.278872       3        1  3.508916
43    -0.328798 -0.410051       4        1  3.472171
42     0.210329 -0.477719       5        1  3.204820
11     0.351812 -0.325405       6        1  2.633569
22    -0.427301 -0.267563       7        1  2.602160
0      0.242617 -0.312864       8        1  2.567839
1     -0.203580  0.211016       9        1  2.523060
10     0.325155  0.242849      10        1  2.506478
18    -0.274287 -0.354314      11        1  2.494808
20    -0.503195 -0.055096      12        1  2.369973
41    -0.398209  0.139703      13        1  2.316628
36     0.114506  0.276025      14        1  2.279737
13    -0.292755 -0.001804      15        1  2.271226
25    -0.326290  0.273551      16        1  2.259279
44    -0.483837  0.227945      17        1  2.167775
7      0.248672  0.373951      18        1  2.070600
49    -0.397543 -0.038645      19        1  2.058986
16     0.117854 -0.255922      20        1  1.948287
47    -0.127719 -0.361373      21        1  1.918693
48     0.013411 -0.511729      22        1  1.889270
4      0.345622 -0.419223      23        1  1.865877
24     0.030510 -0.145929      24        1  1.854953
19     0.029191  0.526260      25        1  1.811130
17     0.104458 -0.474318      26        1  1.781823
28    -0.100940  0.123787      27        1  1.780258
32     0.027238  0.426058      28        1  1.754232
6     -0.192630 -0.488823      29        1  1.748764
31    -0.108204  0.520633      30        1  1.645784
14    -0.279409  0.454003      31        1  1.629500
9      0.055103  0.066131      32        1  1.618897
26     0.531110 -0.076324      33        1  1.599102
21     0.364780 -0.021900      34        1  1.586270
2      0.468533  0.193815      35        1  1.584376
40     0.379610 -0.175153      36        1  1.584314
30     0.238743  0.146219      37        1  1.460245
15    -0.340908 -0.239477      38        1  1.436746
23     0.160245  0.498352      39        1  1.435000
33    -0.064619  0.320252      40        1  1.372291
29     0.474737 -0.021149      41        1  1.357838
37     0.517090  0.100076      42        1  1.349353
38     0.300175  0.448893      43        1  1.332191
12     0.468603 -0.250265      44        1  1.307830
39     0.209226 -0.025965      45        1  1.299633
46    -0.522186  0.077505      46        1  1.286937
45    -0.187931  0.408012      47        1  1.236430
27    -0.387946  0.351816      48        1  1.229458
8     -0.167557 -0.079320      49        1  1.162907
3     -0.500322 -0.179364      50        1  1.160628, topic_info=     Category           Freq         Term          Total  loglift  logprob
term                                                                      
3994  Default  307793.000000     handmade  307793.000000  30.0000  30.0000
2038  Default  498734.000000       cotton  498734.000000  29.0000  29.0000
3687  Default  245764.000000         girl  245764.000000  28.0000  28.0000
513   Default  272846.000000         baby  272846.000000  27.0000  27.0000
2342  Default  222037.000000        decor  222037.000000  26.0000  26.0000
3102  Default  248117.000000       fabric  248117.000000  25.0000  25.0000
380   Default  263499.000000          art  263499.000000  24.0000  24.0000
9812  Default  217081.000000         wood  217081.000000  23.0000  23.0000
2217  Default  207717.000000       custom  207717.000000  22.0000  22.0000
6406  Default  207851.000000        paper  207851.000000  21.0000  21.0000
894   Default  187265.000000         blue  187265.000000  20.0000  20.0000
9666  Default  185639.000000      wedding  185639.000000  19.0000  19.0000
9529  Default  173153.000000      vintage  173153.000000  18.0000  18.0000
3665  Default  528518.000000         gift  528518.000000  17.0000  17.0000
3